In [1]:
%run useful_functions.py

C:\Users\natha\Desktop\17-Cyber-risk_and_the_cross-section_of_stock_returns-main\17-Cyber-risk_and_the_cross-section_of_stock_returns-main\Work_Nathan\useful_functions.py:941: SyntaxWarning: invalid escape sequence '\s'
  item1A = re.sub('\s{2,}',' ', item1A)
C:\Users\natha\Desktop\17-Cyber-risk_and_the_cross-section_of_stock_returns-main\17-Cyber-risk_and_the_cross-section_of_stock_returns-main\Work_Nathan\useful_functions.py:952: SyntaxWarning: invalid escape sequence '\s'
  document = re.sub('\s{2,}',' ', document)
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\natha\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


update 16


In [2]:
import gensim
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import re
from wordfreq import top_n_list
import numpy as np
from nltk.corpus import stopwords
import requests
import requests_random_user_agent
from tqdm.notebook import tqdm
from nltk.stem import PorterStemmer
import pandas as pd
from bs4 import BeautifulSoup
from nltk.tokenize import word_tokenize, sent_tokenize
import dask
dask.config.set(scheduler="processes")
from tqdm.dask import TqdmCallback
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import cosine_similarity
import glob
from gensim.test.utils import datapath
import logging
import random
from random import sample
random.seed(1)
from fpdf import FPDF
from IPython.display import clear_output
import time
import pickle
import random

In [3]:
#read in the previously saved stocknames file
stocknames = pd.read_csv("../../data/stocknames.csv.gz", na_filter = False)
stocknames.replace('', np.nan, inplace = True)

In [4]:
save_path = '../../data/10k_statements_new/financial_text_variables/'
years = np.arange(2020,2025)

common_words = top_n_list('en', 100)
stop_words = stopwords.words('english')
alphabet = re.compile('[^a-z]')

delayed_text = dask.delayed(get_cleaned_text)

list_missing_tokens=[]
for year in years: #years 
    #take the urls of all the 10-K fillings for each year
    urls = stocknames.filter(like = 'url_{}'.format(year))
    urls.index = stocknames.ticker
    urls = urls.dropna()
    
    #compute in batches
    len_batch = 250
    #ceil of euclidian division
    nb_batches = -(len(urls) // -len_batch)
    start = 0
    end = len_batch
    for batch in tqdm(range(nb_batches), desc = 'Cleaning text 10-Ks for {}'.format(year),leave = False):
        for j in range(10): #j attempt to create allpromises list
            #get a list of tokens for each sentence of each document
            allpromises = []
            for ticker, url in urls.iloc[start:end,:].itertuples():
                text=None
                for i in range(10): #i attempt to request text from url
                    try:
                        text = requests.get(url).text
                    except:
                        print("request failed")
                        time.sleep(120)
    
                    if text!=None:
                        break
                        
                if text==None:
                    print("after 10 attempts (20 min), the server still failed to respond, shutting down program...")
                    stop
                            
                        
                allpromises.append(delayed_text(text, ticker = ticker, find_item1A_=True))
    
               
            there_was_no_error=True
            with TqdmCallback(desc = 'Cleaning text 10-Ks in batch', leave = False):
                
                #cleaned_text = dask.compute(allpromises)[0]
                try:
                    cleaned_text = dask.compute(allpromises)[0]
    
    
                except Exception as e:
                            # Generate a random 6-digit number
                            random_number = random.randint(100000, 999999)

                    
                            print("error with tokens, error #"+str(random_number))
                            there_was_no_error=False
                            list_missing_tokens.append(allpromises)
    
                            
                            #with open(str(year)+'missing_tokens_'+str(random_number)+'.pickle', 'wb') as f:
                            #    pickle.dump(list_missing_tokens, f)
    
    
            #save the tokens
            if there_was_no_error:
                for tuple in cleaned_text:

                    
                            ticker_name = tuple[0]
                            list_text = tuple[1]
                            first_indice_1A=tuple[2]
                            last_indice_1A=tuple[3]
                    
                            readability=tuple[4]
                            secret=secrets(list_text)
                            risk_len=risk_section_length(first_indice_1A,last_indice_1A)

                            np.save(save_path+str(year)+"/"+ticker_name+'_readability_secret_risklength.npy', np.array([readability,secret,risk_len]))

                         
                            

                start += len_batch
                end += len_batch
                break

            else:
                print("rebuilding the 'allpromises' list, iteration : "+str(j))
                time.sleep(60)

            if j==9:
                print("impossible to build 'allpromises' after 10 iterations, shutting down program...")
                stop
                        
                

    time.sleep(120)
    
clear_output()
print('Done')

Done


In [5]:
#file:///C:/Users/natha/Desktop/Litterature_Review/Sentiment%20Analysis/sautnerLentVilkovZhang2023.pdf